In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [2]:
import datetime
import pyaurorax
import cv2
import cartopy.crs
import numpy as np

aurorax = pyaurorax.PyAuroraX()
at = aurorax.tools
at.set_theme("dark")

### **Step 1: Download Burst Data, and TREx RGB Skymaps**

In [3]:
# download a minute of TREx RGB Burst data from several sites
dataset_name = "TREX_RGB_RAW_BURST"
start_dt = datetime.datetime(2023, 2, 24, 6, 15)
end_dt = start_dt
site_uid_list = ['fsmi','gill','rabb']
data_download_objs = {}
for site_uid in site_uid_list:
    download_obj = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)
    data_download_objs[site_uid] = download_obj

# Read in the data site-by-site, as we need this separation for mosaicing
data_list = []
for site_uid, download_obj in data_download_objs.items():
    data_list.append(aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames))


# Get list of all skymaps in previous two years up to date of interest for each site
skymap_download_objs = {}
for site_uid in site_uid_list:
    r = aurorax.data.ucalgary.download_best_skymap("TREX_RGB_SKYMAP_IDLSAV", site_uid, start_dt)
    skymap_download_objs[site_uid] = r

# Read and store the last list index skymap for each site, as that is most recent to the date of interest
skymaps = []
for site_uid, download_obj in skymap_download_objs.items():

    skymap = aurorax.data.ucalgary.read(download_obj.dataset, download_obj.filenames[-1])
    skymaps.append(skymap.data[0])

skymaps

[Skymap(project_uid=rgb, site_uid=fsmi, imager_uid=rgb-01, site_map_latitude=60.026112, site_map_longitude=248.067062, ...),
 Skymap(project_uid=rgb, site_uid=gill, imager_uid=rgb-04, site_map_latitude=56.376724, site_map_longitude=265.356323, ...),
 Skymap(project_uid=rgb, site_uid=rabb, imager_uid=rgb-06, site_map_latitude=58.227810, site_map_longitude=256.319366, ...)]

### **Step 2: Manually Resize Burst Data**

The raw TREx RGB Burst data contains images of a larger size compared to the nominal raw data. In order to create a mosaic, we need to resize them to the size of the nominal RGB data (480,553), so that we can use the skymaps for georeferencing.

In [4]:
for i, data in enumerate(data_list):

    # extract the images
    images = data.data
    timestamps = data.timestamp

    resized_images = np.empty((480,553,3,images.shape[-1]))
    resized_timestamps = []

    # resize all images
    for frame_idx in range(images.shape[-1]):
        tmp_frame = np.squeeze(resized_images[:,:,:,frame_idx])
        tmp_frame = cv2.resize(tmp_frame, (553, 480))
        resized_images[:,:,:,frame_idx] = tmp_frame
        resized_timestamps.append(timestamps[frame_idx])

    # manually insert the images back into the data object
    data_list[i].data = resized_images
    data_list[i].timestamp = resized_timestamps

### **Step 3: Build Mosaic in Usual Manner**
Note: If creating a mosaic for a single frame of burst data, it would be computationally advantageous to also chop down the number of images in data, in the previous step, as preparing a full minute of burst data can take some time to process. 

In [5]:
# NOTE: Currently seems to hang, I am guessing because of decimals in timestamps

# # prepare the skymap data
# prepped_skymap = at.mosaic.prep_skymaps(skymaps, 110, n_parallel=3)

# # prepare the image data
# prepped_images = at.mosaic.prep_images(resized_data_list)

# # define the intensity scales for each site
# scale = {
#     "fsmi": [10, 105],
#     "gill": [10, 105],
#     "rabb": [10, 105],
# }

# # create projection
# center_lat = -100.0
# center_lon = 55.0
# projection_obj = cartopy.crs.NearsidePerspective(central_longitude=center_lat, central_latitude=center_lon)

# # create mosaic
# frame_num = 5
# mosaic = at.mosaic.create(prepped_images, prepped_skymap, frame_num, projection_obj, image_intensity_scales=scale)
# print(mosaic)

Preparing skymaps:   0%|          | 0/3 [00:00<?, ?skymap/s]

0
1
2


KeyboardInterrupt: 